In [24]:
# autoreload reloads modules automatically before entering the execution of code typed at the IPython prompt.
%reload_ext autoreload
%autoreload 2
%aimport feature_preprocessing_pipeline
%aimport build_model_by_brand
%aimport build_model
%aimport build_model_for_merged_data
%aimport prepare_data
%aimport bs_lib.bs_preprocess_lib
%aimport constants

In [75]:
from sklearn import set_config
from os import system
from os.path import join, isfile
from joblib import dump, load
from feature_preprocessing_pipeline import categorize, extract_features, clean_variables
from sklearn import tree
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.metrics import make_scorer, mean_squared_error
from build_model_by_brand import evaluate_all_models,evaluate_all_refitted_models, get_set_split,get_best_estimators_params
from build_model import get_one_model_for_all_iterative,evaluate_prediction, get_model
import build_model_for_merged_data as bmmd #import evaluate_model,get_best_estimator
from prepare_data import load_prepared_file
from bs_lib.bs_preprocess_lib import plot_validation_curve,get_learning_curve,plot_learning_curve
import constants as cnst
#from dtreeviz.trees import dtreeviz

In [81]:

from IPython.core.display import display, HTML 
from sklearn.utils import estimator_html_repr 
set_config(display='diagram') 

In [4]:
dataset_directory_path = 'dataset/'
model_directory_path = 'model/'

In [57]:
model_name = 'mercedes_optimize_rmse_2427'
model_filename = f'{model_name}.joblib'
model_path = join(cnst.MODEL_DIR_PATH, model_filename)
model = load(model_path)

In [82]:
X_test = pd.read_csv(join(dataset_directory_path, 'X_test.csv'), index_col=0)
y_test = pd.read_csv(join(dataset_directory_path, 'y_test.csv'), index_col=0)

## Description du model 

### Variables

In [78]:
rfr = model[2]
print(f"Le model {rfr} a {rfr.n_features_in_} variables")

Le model RandomForestRegressor(max_depth=50, max_features=None, min_samples_split=6,
                      n_estimators=10, n_jobs=-1, verbose=False) a 57 variables


### Durée d'entraînement

In [64]:
print(f"Test set:\n X {X_test.shape} y {y_test.shape}")


Test set:
 X (7824, 9) y (7824, 1)


<center><img src="./assets/one_model_by_brand_overview.svg"/></center>

### Traitement des fichiers sources
<center><img src="./assets/file_processing.svg"/></center>

### Préparation des données

<center><img src="./assets/prepare_data.svg"/></center>

#### Suppression ou Imputation des *anomalies*

<center><img src="./assets/outliers.svg"/></center>

### Séparation des données

<center><img src="./assets/split_data.svg"/></center>

### Création du Modèle

<center><img src="./assets/build_model.svg"/></center>

#### Extract Features

Création de nouvelles variables:
1. `age`: $max(year)-year$
1. `model_count`: nombre d'occurence du modèle dans le dataset
1. `occurence_relative`: $\frac{count(model)}{count(allmodel)}$
1. `mileage_per_year`: $\frac{mileage}{age}$
1. `mpg_per_year`: $\frac{mpg}{age}$
1. `tax_per_year`: $\frac{tax}{age}$
1. `engine_size_per_year`: $\frac{enginesize}{age}$
1. `esoteric_composite_ratio`

Suivant la marque de voiture et/ou le modèle de prédiction créé, certaines améliorent le score de prédictions.

#### Transform Features
1. `PolynomialFeature` pour les variables numériques.
1. `KBinsDiscretizer` pour les variables `mpg`, `tax`, `engine_size`, `year`.
1. `OneHotEncoder` ou `OrdinalEncoder` pour les variables `model`, `brand`, `transmission`, `fuel_type`. 

Pour l'`OrdinalEncoder`, les catégories ont été triées relativement à la variable dépendante `price` par **ordre ascendant**.


#### Échelle discrète

##### MPG

<center><img src="./assets/mpg_histplot.png"/></center>

##### Tax

<center><img src="./assets/tax_vs_price_relation_plot.png"/></center>

##### Engine Size

<center><img src="./assets/engine_size_histplot.png"/></center>

### Pipeline

<center><img src="./assets/pipeline.svg"/></center>

### Valeurs des paramètres du pipeline

<center><img src="./assets/best_params_search.svg"/></center>

## Recherche de Modèles

1. Un modèle par marque
1. Un modèle pour toutes les marques
1. Un modèle incrémental pour toutes les marques

### Couches d'optimisation du modèle

<center><img src="./assets/optimization.svg"/></center>

### Par marques

Création d'un modèle par marques

#### Évaluation du modèle de base

In [89]:
evaluate_all_models(search_term='base-model',exclude=['base_score','all_brand'])

PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                       
┏━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃  Brand   ┃ RMSE ┃   R2    ┃ Absolute Mean Error ┃     Model path      ┃
┡━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│   Audi   │ 2866 │ -3.7893 │        1687         │   audi_base-model   │
│   Bmw    │ 2534 │ -4.1015 │        1674         │   bmw_base-model    │
│   Ford   │ 1402 │ -6.4619 │         918         │   ford_base-model   │
│ Hyundai  │ 1585 │ -4.6620 │         891         │ hyundai_base-model  │
│ Mercedes │ 3469 │ -4.2664 │        1803         │ mercedes_base-model │
│   Opel   │ 1146 │ -8.7634 │         785         │   opel_base-model   │
│  Skoda   │ 2743 │ -4.3746 │        1138         │  skoda_base-model   │
│  Toyota  │ 1504 │ -4.0970 │         891         │  toyota_base-model  │
│    Vw    │ 2026 │ -4.6721 │        1191         │    vw_base-model    │
└──────────┴──────┴─────────┴─────────────────────┴─────────────────────┘

#### Optimisation des transformations

In [57]:
evaluate_all_models(search_term='optimize', exclude=['rmse'])

PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                     
┏━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃  Brand   ┃ RMSE ┃   R2    ┃ Absolute Mean Error ┃    Model path     ┃
┡━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│   Audi   │ 1847 │ -3.7893 │        1149         │   audi_optimize   │
│   Bmw    │ 1635 │ -4.1015 │        1108         │   bmw_optimize    │
│   Ford   │ 801  │ -6.4619 │         541         │   ford_optimize   │
│ Hyundai  │ 733  │ -4.6620 │         518         │ hyundai_optimize  │
│ Mercedes │ 2613 │ -4.2664 │        1211         │ mercedes_optimize │
│   Opel   │ 682  │ -8.7634 │         463         │   opel_optimize   │
│  Skoda   │ 1486 │ -4.3745 │         736         │  skoda_optimize   │
│  Toyota  │ 919  │ -4.0970 │         545         │  toyota_optimize  │
│    Vw    │ 1162 │ -4.6721 │         761         │    vw_optimize    │
└──────────┴──────┴─────────┴─────────────────────┴───────────────────┘

#### Optimisation de l'estimateur

In [91]:
# Modèle avec pipeline et estimateur optimisé
#get_best_estimators_params(search_term='optimize_rmse', exclude_term=['estimator'], verbose=True)
evaluate_all_models(search_term='estimator',exclude=['all_brand'])

PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                           
┏━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  Brand   ┃ RMSE ┃   R2    ┃ Absolute Mean Error ┃       Model path        ┃
┡━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   Audi   │ 1567 │ -3.7893 │         871         │   audi_estimator_1567   │
│   Bmw    │ 1635 │ -4.1015 │        1108         │   bmw_estimator_1635    │
│   Ford   │ 784  │ -6.4619 │         486         │   ford_estimator_784    │
│ Hyundai  │ 733  │ -4.6620 │         518         │  hyundai_estimator_733  │
│ Mercedes │ 2073 │ -4.2664 │         983         │ mercedes_estimator_2073 │
│   Opel   │ 586  │ -8.7634 │         376         │   opel_estimator_586    │
│  Skoda   │ 1825 │ -4.3745 │         744         │  skoda_estimator_1825   │
│  Toyota  │ 845  │ -4.0970 │         494         │  toyota_estimator_845   │
│    Vw    │ 947  │ -4.6721 │         558         │    vw_estimator_947     │
└──────────┴──────┴─────────┴─────────────────────┴─────────────────────────┘

#### Optimisation des transformations (avec permutation)

In [92]:
evaluate_all_models(search_term='perm', exclude=[])

PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                      
┏━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃  Brand   ┃ RMSE ┃   R2    ┃ Absolute Mean Error ┃     Model path     ┃
┡━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│   Audi   │ 1792 │ -3.7893 │        1096         │   audi_perm_1792   │
│   Bmw    │ 1656 │ -4.1015 │        1097         │   bmw_perm_1656    │
│   Ford   │ 789  │ -6.4619 │         554         │   ford_perm_789    │
│ Hyundai  │ 760  │ -4.6620 │         536         │  hyundai_perm_760  │
│ Mercedes │ 2266 │ -4.2664 │        1237         │ mercedes_perm_2266 │
│   Opel   │ 707  │ -8.7634 │         461         │   opel_perm_707    │
│  Skoda   │ 1435 │ -4.3745 │         708         │  skoda_perm_1435   │
│  Toyota  │ 916  │ -4.0970 │         580         │  toyota_perm_916   │
│    Vw    │ 1149 │ -4.6721 │         759         │    vw_perm_1149    │
└──────────┴──────┴─────────┴─────────────────────┴────────────────────┘

In [94]:
#get_best_estimators_params(search_term='perm', exclude_term=[],prefix='perm', verbose=True)
evaluate_all_models(search_term='perm-estimator', exclude=[])

PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                                
┏━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  Brand   ┃ RMSE ┃   R2    ┃ Absolute Mean Error ┃          Model path          ┃
┡━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   Audi   │ 1540 │ -3.7893 │         848         │   audi_perm-estimator_1540   │
│   Bmw    │ 1482 │ -4.1015 │         959         │   bmw_perm-estimator_1482    │
│   Ford   │ 785  │ -6.4619 │         507         │   ford_perm-estimator_785    │
│ Hyundai  │ 760  │ -4.6620 │         536         │  hyundai_perm-estimator_760  │
│ Mercedes │ 2266 │ -4.2664 │        1237         │ mercedes_perm-estimator_2266 │
│   Opel   │ 707  │ -8.7634 │         461         │   opel_perm-estimator_707    │
│  Skoda   │ 1435 │ -4.3745 │         708         │  skoda_perm-estimator_1435   │
│  Toyota  │ 862  │ -4.0970 │         503         │  toyota_perm-estimator_862   │
│    Vw    │ 963  │ -4.6721 │         561         │    vw_perm-estimator_963     │
└──────────┴──────┴─────────┴─────────────────────┴──────────────────────────────┘

#### Refit?

In [61]:
# load performance
evaluate_all_refitted_models(base_model_search_term='model', 
                             params_search_term='rmse',
                             model_exclude=['base_score_model.joblib','all_brand'],
                             params_exclude=[])
# Refit toyota_model.joblib using toyota.csv with parameters from toyota_optimize_rmse_919.json

PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                  
┏━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃  Brand   ┃ RMSE ┃   R2    ┃ Absolute Mean Error ┃   Model path   ┃
┡━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│   Audi   │ 2582 │ -3.7893 │        1585         │   audi_model   │
│   Bmw    │ 2473 │ -4.1015 │        1615         │   bmw_model    │
│   Ford   │ 1274 │ -6.4619 │         872         │   ford_model   │
│ Hyundai  │ 1480 │ -4.6620 │         849         │ hyundai_model  │
│ Mercedes │ 3375 │ -4.2664 │        1731         │ mercedes_model │
│   Opel   │ 1107 │ -8.7634 │         756         │   opel_model   │
│  Skoda   │ 2659 │ -4.3746 │        1072         │  skoda_model   │
│  Toyota  │ 1215 │ -4.0970 │         822         │  toyota_model  │
│    Vw    │ 1995 │ -4.6721 │        1165         │    vw_model    │
└──────────┴──────┴─────────┴─────────────────────┴────────────────┘

#### Essais de prédictions

In [95]:
brand = 'opel'
model_name = 'opel_perm-estimator_707'
model_filename = f'{model_name}.joblib'
model_path = join(cnst.MODEL_DIR_PATH, model_filename)
model = load(model_path)
filename = f"{brand}.csv"
df = load_prepared_file(filename=filename)
X_train, X_val, X_test, y_train, y_val, y_test = get_set_split(df, target='price')
evaluate_prediction(model, X_test, y_test,sample=10,add_columns=['model','transmission','fuel_type'])

RMSE: 1030.7270273679117


MODEL PREDICTION TESTING ────────────────────────────────────────────────────────────────────

Prediction results                                                                   
┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Prediction ┃ Real Price ┃ Error ┃ Percentage ┃  model  ┃ transmission ┃ fuel_type ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│   10451    │   10000    │ -451  │    -5 %    │  Astra  │  Semi-Auto   │  Petrol   │
│   13763    │   13481    │ -282  │    -2 %    │  Astra  │    Manual    │  Petrol   │
│    8691    │    8298    │ -393  │    -5 %    │  Mokka  │    Manual    │  Petrol   │
│   10279    │    8290    │ -1989 │   -24 %    │  Astra  │    Manual    │  Petrol   │
│    7968    │    7994    │  26   │    0 %     │  Corsa  │    Manual    │  Petrol   │
│   13959    │   12049    │ -1910 │   -16 %    │ Mokka X │  Semi-Auto   │  Petrol   │
│   10761    │    9959    │ -802  │    -8 %    │  Astra  │  Automatic   │  Diesel   │
│    8664    │    9135    │  471  │    5 %     │  Corsa  │    Manual    │  Petrol   │
│   12697    │   12999    │  302  │    2 %     │  Mokka  │    Manual    │  Petrol   │
│   11474    │   10195    │ -1279 │   -13 %    │ Mokka X │    Manual    │  Petrol   │
└────────────┴────────────┴───────┴────────────┴─────────┴──────────────┴───────────┘

#### Courbe d'apprentissage

In [ ]:

from sklearn.model_selection import StratifiedKFold
mse = make_scorer(mean_squared_error, greater_is_better=False)
skf = StratifiedKFold(n_splits=5,random_state=1, shuffle=True)
plot_learning_curve(model, 'opel', X_train, y_train, axes=None, ylim=None, cv=sk,
                        n_jobs=2, train_sizes=np.linspace(.1, 1.0, 10), show=True,savefig=False)

Le premier graphique montre les courbes d'apprentissage pour l'entraînement et la validation.

Le deuxième graphique montre la durée d'entraînement du modèle relativement à la taille des données d'entraînement. 

Le troisième graphique montre les scores obtenus relativement au temps d'entraînement du modèle.

#### Nombre de variables

In [97]:
rfr = model[2]
print(f"Le model {rfr} a {rfr.n_features_in_} variables")

Le model RandomForestRegressor(max_depth=50, max_features=None, min_samples_split=6,
                      n_estimators=10, n_jobs=-1, verbose=False) a 131 variables


### Toutes marques

1. fusion des données en une seule source de données

In [62]:
bmmd.evaluate_model(model_filename='all_brands.joblib', data_filename='all_brands.csv',verbose=False)

Loading all_brands.joblib


PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                          
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃        Brand         ┃ RMSE ┃   R2    ┃ Absolute Mean Error ┃ Model path ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ Merged Brand Dataset │ 2442 │ -2.8677 │        1220         │ all_brands │
└──────────────────────┴──────┴─────────┴─────────────────────┴────────────┘

In [14]:
bmmd.get_best_estimator(model_filename='all_brands.joblib', data_filename='all_brands.csv',verbose=False)

Loading all_brands.joblib
Model all_brands_estimator_optimized_rmse_2096 saved @ ./model/all_brands_estimator_optimized_rmse_2096.joblib


In [63]:
bmmd.evaluate_model(model_filename='all_brands_estimator_optimized_rmse_2096.joblib',data_filename='all_brands.csv',verbose=False)

Loading all_brands_estimator_optimized_rmse_2096.joblib


PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                                           
┏━━━━━━━━━━━━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃       Brand       ┃ R… ┃  R2  ┃ Absolute Mean Er… ┃              Model path               ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Merged Brand Dat… │ 2… │ -2.… │        960        │ all_brands_estimator_optimized_rmse_… │
└───────────────────┴────┴──────┴───────────────────┴───────────────────────────────────────┘

In [84]:
model = bmmd.get_model(model_filename='all_brands.joblib', data_filename='all_brands.csv',verbose=False)
df = bmmd.get_data('all_brands.csv')
X_train, X_val, X_test, y_train, y_val, y_test = bmmd.get_split_sets(df)
evaluate_prediction(model, X_test, y_test,sample=10,add_columns=['brand','model','transmission','fuel_type'])

Loading all_brands.joblib
RMSE: 3398.386598062283


MODEL PREDICTION TESTING ────────────────────────────────────────────────────────────────────

Prediction results                                                                           
┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Prediction ┃ Real Price ┃ Error ┃ Percentage ┃ brand  ┃  model  ┃ transmission ┃ fuel_ty… ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│   17969    │   17991    │  22   │    0 %     │ toyota │  C-HR   │  Automatic   │  Petrol  │
│   13883    │   11995    │ -1888 │   -16 %    │ toyota │  Yaris  │  Automatic   │  Hybrid  │
│   26001    │   25190    │ -811  │    -3 %    │ skoda  │  Karoq  │  Automatic   │  Diesel  │
│   10959    │   14398    │ 3439  │    24 %    │  opel  │ Mokka X │  Semi-Auto   │  Petrol  │
│   33102    │   26825    │ -6277 │   -23 %    │  bmw   │ 3 Seri… │  Automatic   │  Petrol  │
│   11265    │   18490    │ 7225  │    39 %    │  audi  │   Q2    │  Semi-Auto   │  Petrol  │
│   10313    │   10494    │  181  │    2 %     │ toyota │  Yaris  │    Manual    │  Petrol  │
│   10979    │    9898    │ -1081 │   -11 %    │   vw   │ Golf SV │    Manual    │  Diesel  │
│   34515    │   36984    │ 2469  │    7 %     │  audi  │   Q5    │  Automatic   │  Diesel  │
│   28188    │   28919    │  731  │    3 %     │  bmw   │ 5 Seri… │  Semi-Auto   │  Diesel  │
└────────────┴────────────┴───────┴────────────┴────────┴─────────┴──────────────┴──────────┘

### Entraînement Incrémental

Le model est entrainé sur chaque jeu de données en conservant les arbres précédemment créés.

In [73]:
iterative_model = get_one_model_for_all_iterative(model_to_dump=True, evaluation=True, verbose=False)

PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                           
┏━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Brand    ┃ RMSE ┃   R2   ┃ Absolute Mean Error ┃       Model path       ┃
┡━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Inc. Model │ 6930 │ 0.5995 │        4163         │ all_brand_model_6930.0 │
└────────────┴──────┴────────┴─────────────────────┴────────────────────────┘

In [74]:
bmmd.get_best_estimator(model_filename='all_brand_model_6930.0.joblib', data_filename='all_brands.csv',verbose=False)

Loading all_brand_model_6930.0.joblib
Model all_brand_model_6930_estimator_optimized_rmse_1642 saved @ ./model/all_brand_model_6930_estimator_optimized_rmse_1642.joblib


In [76]:
iterative_model = get_model(model_path_to_load='./model/all_brand_model_6930_estimator_optimized_rmse_1642.joblib')
df = bmmd.get_data('all_brands.csv')
X_train, X_val, X_test, y_train, y_val, y_test = bmmd.get_split_sets(df)

In [81]:
bmmd.evaluate_model(model_filename='all_brand_model_6930_estimator_optimized_rmse_1642.joblib', data_filename='all_brands.csv',verbose=False)

Loading all_brand_model_6930_estimator_optimized_rmse_1642.joblib


PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                                       
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Brand         ┃ RMSE ┃   R2    ┃ Absolute Mean Error ┃       Model path        ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Merged Brand Dataset │ 1642 │ -2.8677 │         651         │ all_brand_model_6930... │
└──────────────────────┴──────┴─────────┴─────────────────────┴─────────────────────────┘

In [83]:
evaluate_prediction(iterative_model, X_test, y_test,sample=10,add_columns=['brand','model','transmission','fuel_type'])

RMSE: 2712.2443298190624


MODEL PREDICTION TESTING ────────────────────────────────────────────────────────────────────

Prediction results                                                                           
┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Prediction ┃ Real Price ┃ Error ┃ Percentage ┃ brand  ┃  model  ┃ transmission ┃ fuel_ty… ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│   20653    │   17991    │ -2662 │   -15 %    │ toyota │  C-HR   │  Automatic   │  Petrol  │
│   13509    │   11995    │ -1514 │   -13 %    │ toyota │  Yaris  │  Automatic   │  Hybrid  │
│   26106    │   25190    │ -916  │    -4 %    │ skoda  │  Karoq  │  Automatic   │  Diesel  │
│   12664    │   14398    │ 1734  │    12 %    │  opel  │ Mokka X │  Semi-Auto   │  Petrol  │
│   33800    │   26825    │ -6975 │   -26 %    │  bmw   │ 3 Seri… │  Automatic   │  Petrol  │
│   17602    │   18490    │  888  │    5 %     │  audi  │   Q2    │  Semi-Auto   │  Petrol  │
│   10383    │   10494    │  111  │    1 %     │ toyota │  Yaris  │    Manual    │  Petrol  │
│   10984    │    9898    │ -1086 │   -11 %    │   vw   │ Golf SV │    Manual    │  Diesel  │
│   36625    │   36984    │  359  │    1 %     │  audi  │   Q5    │  Automatic   │  Diesel  │
│   25824    │   28919    │ 3095  │    11 %    │  bmw   │ 5 Seri… │  Semi-Auto   │  Diesel  │
└────────────┴────────────┴───────┴────────────┴────────┴─────────┴──────────────┴──────────┘